<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Модель:-Логистическая-регрессия" data-toc-modified-id="Модель:-Логистическая-регрессия-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Модель: Логистическая регрессия</a></span></li><li><span><a href="#Модель:LGBMClassifier" data-toc-modified-id="Модель:LGBMClassifier-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Модель:LGBMClassifier</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Проверка-на-тесте" data-toc-modified-id="Проверка-на-тесте-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Проверка на тесте</a></span><ul class="toc-item"><li><span><a href="#Модель:-Логистическая-регрессия" data-toc-modified-id="Модель:-Логистическая-регрессия-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Модель: Логистическая регрессия</a></span></li><li><span><a href="#Модель:LGBMClassifier" data-toc-modified-id="Модель:LGBMClassifier-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Модель:LGBMClassifier</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [3]:
import pandas as pd 
import numpy as np 

from sklearn.pipeline import Pipeline

import re
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer

from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier 
from catboost import CatBoostClassifier

from sklearn.metrics import f1_score

import time
import warnings
warnings.filterwarnings('ignore')

nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
df = pd.read_csv("/datasets/toxic_comments.csv") #Выведем таблицу на экран
df

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159566,""":::::And for the second time of asking, when ...",0
159567,You should be ashamed of yourself \n\nThat is ...,0
159568,"Spitzer \n\nUmm, theres no actual article for ...",0
159569,And it looks like it was actually you who put ...,0


In [5]:
data.info(memory_usage='deep') #Посмотрим информацию о таблице

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 80.1 MB


In [6]:
df['toxic'].value_counts()

0    143346
1     16225
Name: toxic, dtype: int64


<div style="border:solid green 2px; padding: 10px">
    
159571 строка
<br>Дубликатов и пропусков нет
<br>2 столбца: toxic, text
<br>90% твитов не токсичны

In [12]:
def clean_text(text): #Функция для обработки английского текста
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    text = text.strip(' ')
    return text

In [13]:
%%time
df['text'] = df['text'].map(lambda x: clean_text(x))

CPU times: user 13.1 s, sys: 73.7 ms, total: 13.2 s
Wall time: 13.2 s


In [14]:
df

,text,toxic
0,explanation why the edits made under my userna...,0
1,d aww he matches this background colour i am s...,0
2,hey man i am really not trying to edit war it ...,0
3,more i cannot make any real suggestions on imp...,0
4,you sir are my hero any chance you remember wh...,0
...,...,...
159566,and for the second time of asking when your vi...,0
159567,you should be ashamed of yourself that is a ho...,0
159568,spitzer umm theres no actual article for prost...,0
159569,and it looks like it was actually you who put ...,0


### Вывод

Данные подготовлены и обработаны. В данных 2 столбца (toxic, text), 159571 строка. Дубликатов и пропусков нет. 90% твитов не токсичны.


## Обучение

In [15]:
X = df.drop('toxic', axis=1)
y = df['toxic']

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=12345)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(111699, 1)
(47872, 1)
(111699,)
(47872,)


### Модель: Логистическая регрессия

In [20]:
%%time

lr_pipe = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1,3), min_df=3, max_df=0.9, use_idf=1,
               smooth_idf=1, sublinear_tf=1, stop_words=stopwords)),
    ('clf', LogisticRegression(random_state=12345))])

params = {'clf__C': [0.1, 1, 10, 100],
          'clf__class_weight': ['balanced', None]}

lr_grid = GridSearchCV(estimator=lr_pipe, param_grid=params, cv=3, scoring='f1', n_jobs=-1, refit=False)
lr_grid.fit(X_train['text'], y_train)
lr_best_paramms = lr_grid.best_params_

print(lr_best_paramms)
print(lr_grid.best_score_)

{'clf__C': 10, 'clf__class_weight': 'balanced'}
0.7757764993912689
CPU times: user 12min 56s, sys: 10min 55s, total: 23min 52s
Wall time: 23min 53s


### Модель:LGBMClassifier

In [21]:
%%time

lgb_pipe = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1,3), min_df=3, max_df=0.9, use_idf=1,
               smooth_idf=1, sublinear_tf=1, stop_words=stopwords)),
    ('clf', LGBMClassifier(random_state=12345))])

params = {
  'clf__n_estimators': [200],
  'clf__learning_rate': [0.15, 0.25],
  'clf__max_depth': [ 10, -1]}

lgb_grid = GridSearchCV(estimator=lgb_pipe, param_grid=params, cv=3, scoring='f1', n_jobs=-1, refit=False)
lgb_grid.fit(X_train['text'], y_train)
lgb_best_params = lgb_grid.best_params_

print(lgb_best_params)
print(lgb_grid.best_score_)

{'clf__learning_rate': 0.25, 'clf__max_depth': -1, 'clf__n_estimators': 200}
0.7686050343448693
CPU times: user 5h 24min 52s, sys: 2min 37s, total: 5h 27min 29s
Wall time: 5h 29min


### Вывод

LogisticRegression on train: 0.7757764993912689
<br>LGBMClassifier on train: 0.7686050343448693

## Проверка на тесте

In [22]:
vectorize = TfidfVectorizer(ngram_range=(1,3),
               min_df=3, max_df=0.9, use_idf=1,
               smooth_idf=1, sublinear_tf=1, stop_words=stopwords)

In [23]:
X_train = vectorize.fit_transform(X_train['text'])
X_test = vectorize.transform(X_test['text'])

In [25]:
def scoring(fitted_model):
    test_pred = fitted_model.predict(X_test)
    test_f1 = f1_score(y_test, test_pred)
    
    print('F1 on test: {:.3f}'.format(test_f1))

### Модель: Логистическая регрессия

In [28]:
lr_m = LogisticRegression(C=10, class_weight='balanced', random_state=12345)
lr_m.fit(X_train, y_train)
scoring(lr_m)

F1 on test: 0.784


### Модель:LGBMClassifier

In [27]:
lgb_m = LGBMClassifier(learning_rate=0.25, max_depth=-1, n_estimators=200, random_state = 12345)
lgb_m.fit(X_train, y_train)
scoring(lgb_m)

F1 on test: 0.778


### Вывод

LogisticRegression on test: 0.784
<br>LGBMClassifier on test: 0.778

## Выводы

<div style="border:solid green 5px; padding: 20px">
    
**<font color="green">Подготовка</font>**
<br> Данные подготовлены и обработаны. В данных 2 столбца (toxic, text), 159571 строка. Дубликатов и пропусков нет. 90% твитов не токсичны.
    
**<font color="green">Обучение</font>**
<br>LogisticRegression on train: 0.7757764993912689
<br>LGBMClassifier on train: 0.7686050343448693
    
**<font color="green">Проверка на тесте</font>**
<br>LogisticRegression on test: 0.784
<br>LGBMClassifier on test: 0.778
    
    